### MDE

Минимальный ожидаемый эффект — это наименьший истинный эффект полученный от изменений, который с уверенностью сможет обнаружить статистический критерий.
Если прокраса в метрике нет, то это не означает отсутствие эффекта. Эффект может и есть, но он точно не больше, чем MDE для α, β и дисперсии.

Допустим, мы запустили A/B. Было накоплено по ~500 наблюдений в каждой ветке эксперимента. Мы хотим проверить метрику с непрерывным распределением и узнать какой “точности” мы добились за это время.

In [9]:
import numpy as np
from scipy.stats import norm
from statsmodels.stats.power import tt_ind_solve_power 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def estimate_effect_size(sd, n, alpha=0.05, power=0.8):
    """
    Расчет MDE для баланса 50/50
    :param sd: ско одной группы
    :param n: размер выбрки в одной группе
    :return: MDE
    """
    S = np.sqrt((sd**2 / n) + (sd**2 / n))
    M = norm.ppf(q=1-alpha/2) + norm.ppf(q=power)
    return M * S

In [3]:
n = 500
sd = 1
print(estimate_effect_size(sd, n))

0.17718780696593192


Мы получили в качестве результата 0.17. Это означает, что эффекты ниже этого значения будут обладать меньшей мощностью. Соответственно, вероятность того, что эффект действительно есть будет меньше. Эффекты выше этого значения, наоборот, будут мощнее.

Теперь сделаем обратную операцию и рассчитаем количество наблюдений, зная Effect Size. При тех же значениях alpha и power, получим тот же самый размер выборки, что был ранее в качестве входного параметра:

In [6]:
def estimate_sample_size(effect_size, sd, alpha=0.05, power=0.8):
    """
    Расчет N для баланса 50/50
    :param sd: ско одной группы
    :param effect_size: ожидания по изменения в метрику
    :return: N
    """
    M = (norm.ppf(q=1-alpha/2) + norm.ppf(q=power))**2
    return 2 * M * sd**2 / effect_size**2 

In [8]:
effect_size = 0.17718780696593192
sd = 1
print(estimate_sample_size(effect_size, sd))

500.0


Ранее было сказано, что эффекты ниже 0.17 будут обладать меньшей мощностью при 500 наблюдениях. Следовательно, больше наблюдений → больше мощность для того же эффекта и выше.

In [12]:
alpha = 0.05
power = 0.8
n = 500
sd = 1
effect_size = 0.17718780696593192

print(f"""
estimate_sample_size: {estimate_sample_size(effect_size, sd)}
estimate_effect_size: {estimate_effect_size(sd, n)}
tt_ind_solve_power nobs: {tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=1)}
tt_ind_solve_power effect_size: {tt_ind_solve_power(nobs1=n, alpha=alpha, power=power, ratio=1)}
""")


estimate_sample_size: 500.0
estimate_effect_size: 0.17718780696593192
tt_ind_solve_power nobs: 500.96095684136526
tt_ind_solve_power effect_size: 0.17735842307242328

